In [1]:
# ! git clone https://github.com/jlevy44/PairedHE2Tri

Cloning into 'PairedHE2Tri'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 265 (delta 75), reused 244 (delta 54), pack-reused 0
Receiving objects: 100% (265/265), 8.08 MiB | 32.32 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Checking out files: 100% (174/174), done.


In [2]:
! ls PairedHE2Tri/

CycleGAN.ipynb	 docs		  npy2dzi.html	 parallel_npy2dzi.py  train.py
LICENSE		 environment.yml  npy2dzi.py	 pix2pix.ipynb	      util
README.md	 imgs		  npy2tif.py	 requirements.txt
all.log		 log2		  openseadragon  scripts
convert_orig.py  models		  options	 test.py
data		 nohup.out	  p_scripts	 test_wsi.py


In [9]:
import fire

import numpy as np
import PIL
from PIL import Image

import os
import sys
import deepzoom
import time
import shutil
import subprocess
import glob

import os
import time
import subprocess
from multiprocessing import Process

import fire

import numpy as np
import PIL
from PIL import Image

import os
import sys
import deepzoom
import time
import shutil

def worker(png_path, web_dir):
    # disable safety checks for large images
    PIL.Image.MAX_IMAGE_PIXELS = None

    name=os.path.basename(png_path).replace(".png","")

    prefix_path = os.path.join(web_dir, "images", name)
    dzi_path = prefix_path + ".dzi"
    base_html_path = "PairedHE2Tri/npy2dzi.html"
    new_html_path = os.path.join(web_dir, "index.html")

    openseadragon_src = "PairedHE2Tri/openseadragon/"
    openseadragon_dst = os.path.join(web_dir, "openseadragon/")

    title=name

    # create dzi files
    START_TIME = time.time()
    print("Creating .dzi file")
    creator = deepzoom.ImageCreator(
        tile_size=256,
        tile_overlap=0,
        tile_format="png",
        image_quality=1.0,
    )
    creator.create(png_path, dzi_path)
    print("Execution time (s):", time.time() - START_TIME)
    print("Done.\n")

    START_TIME = time.time()
    print("Creating HTML files")
    # create html files
    with open(base_html_path, "r") as f:
        HTML_STR = "".join(f.readlines())
    HTML_STR = HTML_STR.replace("{REPLACE_wsi_prefix}", os.path.join("images", name))
    HTML_STR = HTML_STR.replace("{REPLACE_title}", title)
    with open(new_html_path, "w") as f:
        f.write(HTML_STR)
    # copy openseadragon
    if not os.path.isdir(openseadragon_dst):
        shutil.copytree(openseadragon_src, openseadragon_dst)
    print("Execution time (s):", time.time() - START_TIME)
    print("Done.\n")


def png2dzi(image_path,web_dir,basename):
    PROGRAM_START_TIME = time.time()

    jobs = []
    for png_path in glob.glob(f"{image_path}/{basename}*.png"):
        p = Process(target=worker, args=(png_path, web_dir))
        jobs += [p]
        p.start()

# def png2dzi(image_dir, web_dir, basename, shrink_factor=1): # image_dir is where all images are, ordered
#     # disable safety checks for large images
#     PIL.Image.MAX_IMAGE_PIXELS = None

# #     assert(wsi_name[-4:] == ".npy")

# #     wsi_prefix = wsi_name[:-4]

#     openseadragon_src = "PairedHE2Tri/openseadragon/"
#     openseadragon_dst = os.path.join(web_dir, "openseadragon/")

#     for png_path in glob.glob(os.path.join(image_dir,f"{basename}*.png")):
        
#         name=os.path.basename(png_path).replace(".png","")

#         prefix_path = os.path.join(web_dir, "images", name)
#         dzi_path = prefix_path + ".dzi"
#         base_html_path = "PairedHE2Tri/npy2dzi.html"
#         new_html_path = os.path.join(web_dir, "index.html")
        
# #         iter_name = web_dir[web_dir.rindex("/") + 1:]
#         title = name#iter_name + " " + wsi_name

#         # create dzi files
#         START_TIME = time.time()
#         print("Creating .dzi file")
#         creator = deepzoom.ImageCreator(
#             tile_size=256,
#             tile_overlap=0,
#             tile_format="png",
#             image_quality=1.0,
#         )
#         creator.create(png_path, dzi_path)
#         print("Execution time (s):", time.time() - START_TIME)
#         print("Done.\n")

#         START_TIME = time.time()
#         print("Creating HTML files")
#         # create html files
#         with open(base_html_path, "r") as f:
#             HTML_STR = "".join(f.readlines())
#         HTML_STR = HTML_STR.replace("{REPLACE_wsi_prefix}", os.path.join("images", name))
#         HTML_STR = HTML_STR.replace("{REPLACE_title}", title)
#         with open(new_html_path, "w") as f:
#             f.write(HTML_STR)
#     # copy openseadragon
#     if not os.path.isdir(openseadragon_dst):
#         shutil.copytree(openseadragon_src, openseadragon_dst)
#     print("Execution time (s):", time.time() - START_TIME)
#     print("Done.\n")

# if __name__=="__main__":
#     fire.Fire(main)

In [17]:
!ls bcc/web_test bcc/web_test/openseadragon

bcc/web_test:
images	index.html  openseadragon

bcc/web_test/openseadragon:
LICENSE.txt    images		 openseadragon.js.map  openseadragon.min.js.map
changelog.txt  openseadragon.js  openseadragon.min.js


In [26]:
# ! rm bcc/web_test/images -r

In [27]:
import dask
png2dzi("bcc/interactive_test", "bcc/web_test", "140_A1c")

Creating .dzi file
Creating .dzi file
Creating .dzi file
Creating .dzi file
Creating .dzi file
Creating .dzi file
Execution time (s): 205.57382822036743
Done.

Creating HTML files
Execution time (s): 0.01861715316772461
Done.

Execution time (s): 211.82322072982788
Done.

Creating HTML files
Execution time (s): 0.007509708404541016
Done.

Execution time (s): 271.32965755462646
Done.

Creating HTML files
Execution time (s): 0.011391878128051758
Done.

Execution time (s): 280.06063985824585
Done.

Creating HTML files
Execution time (s): 0.007261991500854492
Done.

Execution time (s): 339.9956157207489
Done.

Creating HTML files
Execution time (s): 0.00972890853881836
Done.

Execution time (s): 346.4643168449402
Done.

Creating HTML files
Execution time (s): 0.0076694488525390625
Done.



In [ ]:
# slow, want to use vips in parallel

In [22]:
! ls bcc/web_test/images/

140_A1c.img.0.dzi    140_A1c.img.2.dzi	       140_A1c.prob_map.1.dzi
140_A1c.img.0_files  140_A1c.img.2_files       140_A1c.prob_map.1_files
140_A1c.img.1.dzi    140_A1c.prob_map.0.dzi    140_A1c.prob_map.2.dzi
140_A1c.img.1_files  140_A1c.prob_map.0_files  140_A1c.prob_map.2_files


In [29]:
# generate final index file
# make an index of indexes w/ database
# add autoselect probability of other one? so only 1 list not two??
txt='''<html>
<head>
    <link rel="stylesheet" href="style.css">
    <title id="title">140_A1c</title>
</head>
<body>
    <div class="float-container">
        <div>
            <span>Modes</span>
            <button id="curtainButton">Curtain</button>
            <button id="syncButton">Sync</button>
        </div>
        Image 1: &nbsp <select id="tileSourceSelect1"></select> <br>
        Image 2: <select id="tileSourceSelect2"></select> <br>
        <div class="float-child float-left">
            <span class="image-title" id="leftImageText"></span> <br>
        </div>
        <div class="float-child float-right">
            <span class="image-title" id="rightImageText"></span> <br>
        </div>
    </div>
    <div class="openseadragon" id="viewer"></div>
    <script src="scripts.js"></script>
    <script src="/openseadragon/openseadragon.min.js"></script>
    <script src="/openseadragon/openseadragon-curtain-sync.min.js"></script>
    <script type="text/javascript">

        var imagePrefix = 0;

        var allTileSources = [];
        for (var i = 0; i <= 2; i += 1) {
            allTileSources.push("images/140_A1c.img."+i+".dzi");
            allTileSources.push("images/140_A1c.prob_map."+i+".dzi");
        }
        console.log(allTileSources);
        document.getElementById("title").innerHTML = "140_A1c";

        var viewerImages = [];
        for (var i = 0; i < allTileSources.length; i++) {
            viewerImages.push({
                key: allTileSources[i],
                tileSource: allTileSources[i],
            });
        }
        // create viewers
        var viewer = new CurtainSyncViewer({
            mode: 'sync',
            container: document.getElementById("viewer"),
            images: viewerImages,
        });

        var curtainButton = document.getElementById("curtainButton");
        var syncButton = document.getElementById("syncButton");
        curtainButton.onclick = function() {
            viewer.setMode("curtain");
        };
        syncButton.onclick = function() {
            viewer.setMode("sync");
        };



        // setup drop-down menus
        var tileSourceSelect1 = document.getElementById("tileSourceSelect1");
        var tileSourceSelect2 = document.getElementById("tileSourceSelect2");
        for (var i = 0; i < allTileSources.length; i++) {
            tileSourceSelect1.appendChild(new Option(allTileSources[i], allTileSources[i]));
            tileSourceSelect2.appendChild(new Option(allTileSources[i], allTileSources[i]));
        }

        // default images shown
        var image1 = allTileSources[0];
        var image2 = allTileSources[1];
        viewer.setImageShown(image1, true)
        viewer.setImageShown(image2, true)
        tileSourceSelect1.value = image1;
        tileSourceSelect2.value = image2;
        var leftImageText = document.getElementById("leftImageText");
        var rightImageText = document.getElementById("rightImageText");
        leftImageText.innerHTML = image1;
        rightImageText.innerHTML = image2;

        function setImageOrderText() {
            if (allTileSources.indexOf(image1) < allTileSources.indexOf(image2)) {
                leftImageText.innerHTML = image1;
                rightImageText.innerHTML = image2;
            } else {
                leftImageText.innerHTML = image2;
                rightImageText.innerHTML = image1;
            }
        }

        // setup dropdown menu actions
        tileSourceSelect1.onchange = function(a) {
            console.log("Left image:", this.value);
            if (this.value != image1 && this.value != image2) {
                var curZoom = viewer.getZoom();

                viewer.setImageShown(image1, false);
                image1 = this.value;
                viewer.setImageShown(image1, true);
                setTimeout(function() {
                    viewer.setZoom(curZoom);
                }, 100);

                setImageOrderText();
            } else {
                console.log("EXCEPTION: Selected image is already displayed.");
            }
        }
        tileSourceSelect2.onchange = function(a) {
            console.log("Image:", this.value);
            if (this.value != image1 && this.value != image2) {
                var curZoom = viewer.getZoom();

                viewer.setImageShown(image2, false);
                image2 = this.value;
                viewer.setImageShown(image2, true);
                setTimeout(function() {
                    viewer.setZoom(curZoom);
                }, 100);

                setImageOrderText();
            } else {
                console.log("EXCEPTION: Selected image is already displayed.");
            }
        }

    </script>
</body>
</html>'''

In [30]:
with open('bcc/web_test/index.html','w') as f:
    f.write(txt)